In [1]:
import pennylane as qml
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

In [2]:
# Create a toy dataset
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Standardize the data
print(y_train.shape)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(800,)


In [3]:
print(X_train)

[[ 1.76755973  0.05440598]
 [-1.76617114  0.31537521]
 [-0.72279926  1.65748645]
 ...
 [-1.46428304  0.67329721]
 [ 0.66197793 -0.22513462]
 [ 1.64081996 -0.23154454]]


In [4]:
n_qubits = 2  # Number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(x, weights):
    # Angle embedding
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))

    # Basic entangler layers
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    
    # Measurement
    return qml.expval(qml.PauliZ(0))


In [5]:
import tensorflow as tf

def cost(weights, X, y):
    predictions = []
    
    # Gather predictions from the circuit
    for x in X:
        pred = circuit(x, weights)  # Get the prediction for each input
        predictions.append(pred)

    # Convert predictions to a NumPy array
    predictions = np.array(predictions)

    # Ensure predictions are in the range [0, 1]
    predictions = (predictions + 1) / 2  # Map from [-1, 1] to [0, 1]

    # Calculate Binary Cross-Entropy loss
    # Using numpy's clip function to avoid log(0)
    epsilon = 1e-12  # Small constant to avoid division by zero
    predictions = np.clip(predictions, epsilon, 1 - epsilon)  # Clip values

    # BCE calculation
    bce_loss = -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))

    return bce_loss
# Initialize weights randomly (3 layers with 3 parameters each for 2 qubits)
n_layers = 1
weights = np.random.randn(n_layers, 2, 3)  # (layers, rotations, qubits)




In [6]:
circuit_draw = qml.draw(circuit)(X_train[0], weights)  # Visualize with the first training sample
print(circuit_draw)


0: ─╭AngleEmbedding(M0)─╭StronglyEntanglingLayers(M1)─┤  <Z>
1: ─╰AngleEmbedding(M0)─╰StronglyEntanglingLayers(M1)─┤     

M0 = 
[1.76755973 0.05440598]
M1 = 
[[[-0.32022268  0.66973221 -0.25760099]
  [ 0.5032784   1.50313558  0.87035659]]]


In [7]:
from scipy.optimize import minimize

# Optimize the weights
print(weights.flatten())
result = minimize(lambda w: cost(w.reshape(n_layers, 2, 3), X_train, y_train), 
                  weights.flatten(), method='BFGS')
optimized_weights = result.x.reshape(n_layers, 2, 3)



[-0.32022268  0.66973221 -0.25760099  0.5032784   1.50313558  0.87035659]


In [8]:
def predict(X):
    predictions = [circuit(x, optimized_weights) for x in X]
    predictions = np.array(predictions)
    return np.round((predictions + 1) / 2)  # Convert to binary (0 or 1)

# Make predictions
y_pred = predict(X_test)

# Check the shape of predictions
print(f"Predictions shape: {y_pred.shape}")
print(y_pred)

Predictions shape: (200,)
[1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 0. 1. 0. 1. 1. 1. 1.]


In [9]:
print(y_pred,y_test)
accuracy_score(y_pred,y_test)

[1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 0. 1. 0. 1. 1. 1. 1.] [1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 1 1
 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0 0 1 0 0 1
 1 1 1 0 0 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 1 1 0 0 1 0 0 1 1
 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0
 1 0 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 1 1
 0 0 0 1 1 0 1 

0.855

In [10]:
import numpy as np
from sklearn.metrics import log_loss

# Sample true labels (0s and 1s for binary classification)
y_true = np.array([0, 1, 1, 0])

# Sample predicted probabilities
# These should be probabilities (between 0 and 1)
y_pred = np.array([0.1, 0.9, 0.8, 0.3])  # Predicted probabilities for the positive class

# Calculate cross-entropy loss
loss = log_loss(y_true, y_pred)

print(f"Cross-Entropy Loss: {loss:.4f}")

Cross-Entropy Loss: 0.1976
